In [ ]:
import csv
from pathlib import Path
import PC_Fns

Error: Invalid response: 500 Internal Server Error

In [6]:
wkdir = Path(r"C:\Users\ucasbwh\Desktop\bin\PROC\PANCAM_HRC_Calibration_HEAT_ON")
sci_file = wkdir / "PANCAM_HRC_Calibration_HEAT_ON_SC.txt"
img_raw_dir = wkdir / "PROC" / "IMG_RAW"

num_pkts = sum(1 for line in open(sci_file))
print(num_pkts)
num_imgs = num_pkts // 7

7


In [7]:
img_lst = list(range(num_imgs))
f = img_lst
for item in img_lst:
    write_file = img_raw_dir / (str(item).zfill(2) + ".pci_raw")
    f[item] = open(write_file, 'w+b')

tmp_file = img_raw_dir / "SC_Bin_Comp.bin"
f_tmp = open(tmp_file, 'w+b')

pkts = 7
cur_img = 0
cur_pkt = 1
with open(sci_file) as sci:
    line = sci.readline()
    while line != "":
        line_red = line.replace("Timestamp: ", "")
        # Remove TimeStamp and [Informative]
        line_red = line_red[26:]
        line_red = line_red.replace("0x", "")
        line_red = line_red.replace("\n", "").replace("\r", "")
        binary_format = bytes.fromhex(line_red)
        f_tmp.write(binary_format)
        data = binary_format[12:-1]
        if cur_pkt < 7:
            if len(data) != 307200:
                print("Warning LDT length incorrect")

        elif cur_pkt == 7:
            if len(data) != 254000:
                print("Warning last LDT length incorrect: ", len(data))
        f[cur_img].write(data)
        # Limit to writing 7 packets to each binary file
        if cur_pkt == pkts:
            cur_img += 1
            cur_pkt = 1
        else:
            cur_pkt += 1
        line = sci.readline()

for file in f:
    file.close()

f_tmp.close()

Warning last LDT length incorrect:  297152


In [17]:
# Recreate the binary file and then compare them

ref = wkdir / "PANCAM_WAC_RR_SC.bin"

if filecmp.cmp(tmp_file, ref, shallow=False):
    print("Files Match")
else:
    print("Files do not Match")

Files Match


In [21]:
sci_files = PC_Fns.Find_Files(
            wkdir, '*SC.txt', SingleFile=False, Recursive=False)
print(sci_files)

[WindowsPath('C:/Users/ucasbwh/Desktop/WAC_RR Test/PANCAM_WAC_RR_SC.txt')]
